# Model 2

This notebboks shows a basic usage example of the `Model2` for the railway scheduling problem. This model solves the railwayscheduling problem using the branch & bounds algorithm implemented in Gurobi with the addition of the initial solution generation through the simulated annealing algorith, the usage of additional valid inequalities to tighted the feasible space and the inclusion of the following cutting plane generators:

- Boolean quadratic polytope cuts
- Clique cuts
- Cover cuts
- Flow cover cuts
- Flow path cuts
- Gomori cuts
- GUB cover cuts
- Implied bound cuts
- Lift-and-project cuts
- MIR cuts
- Mod-k cuts
- Network cuts
- Relax-and-lift cuts
- Strong-CG cuts
- $\{0, \frac{1}{2}\}$ cuts

To initialize the model the following imports are needed.

In [1]:
import os
import sys
import numpy as np
from railway import *

We then change the directory to the root folder of the project in order to correctly load the datasets.\
⚠ **WARNING**: change the path below, which might differ from mine, according to your file system setting in order to correctly load the datasets.

In [2]:
# Root folder directory
ROOT_DIR = "/home/marco/railway-scheduling"

# Set the current directory to root directory
os.chdir(ROOT_DIR)
sys.path.append(os.getcwd())
print(f"Current working directory: 📂 {os.getcwd()}")

Current working directory: 📂 /home/marco/railway-scheduling


We can then initialize the railway scheduling problem by loading one of the datasets present in the `datasets/` folder. To do so we need to define the parameters:

- `N`: number of nodes (stations), with $N \in \{10, 20, 40\}$
- `T`: time horizon, with $T \in \{10, 50, 100\}$
- `J`: number of jobs, with $J \in \{10, 40, 80\}$
- `P`: total number of passengers per arc, currently only $P = 1000$ is available
- `K`: number of alternative routes, currently only $K = 3$ is available

And the problem can be instantiated as follows. Notice that we use the same class as in the `Model0` notebook, the difference will be in which methods we call before the optimization step.

In [4]:
# Define problem parameters
N = 10
T = 10
J = 10
P = 1000
K = 3

# Name of the file to load
FILENAME = f"datasets/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"

# Instantiate the Railway object
rail = Railway.load(FILENAME)

# Print the model in summary
print(rail)

Railway scheduling problem

Parameters:
N:  10 stations
T:  10 periods
J:  10 jobs
P:  1000 passengers
K:  3 alternative routes
Aj: 10 jobs with arcs
Ja: 10 arcs with jobs
C:  0 arcs unavailable simultaneously

Optimization model:
Variables:   0
Constraints: 0
Objective:   0.0
Status:      LOADED



Alternatively the model can be initialized using the default constructor rather than importing it from a dataset file:

```python
rail = Railway(N, T, J, P, K)
```

But then the other problem parameters such as stations coordinates or passengers' demands must be randomly generated with the `generate()` method. See the `apps/generate.py` file for an example on how to do that.

An initial solution can then be generated and set using the simulated annealing meta-heuristic as follows.

In [5]:
# Generate initial solution through simulated annealing
print('Running simulated annealing (SA)...')
SA_objective, SA_time = rail.simulated_annealing(
    T=5e3,
    c=0.99,
    L=1,
    min_T=1,
    max_time=30,
    debug=True
)

# Set the initial solution
rail.set_solution()

# Print initial solution
print(f"Initial solution with SA:\t{rail.S}")
print(f"Initial SA objective:\t{SA_objective}")

Running simulated annealing (SA)...
Time:  28.4 s, Iteration:   848, T:    0.99, Current Obj.:    3913, Best Obj.:    3913	 
Initial solution with SA:	{1: 1, 2: 7, 3: 4, 4: 1, 5: 1, 6: 3, 7: 7, 8: 1, 9: 4, 10: 8}
Initial SA objective:	{1: 1, 2: 7, 3: 4, 4: 1, 5: 1, 6: 3, 7: 7, 8: 1, 9: 4, 10: 8}


Then, we can set the problem parameters, constraints and the objective function to minimize with the dedicated methods.

In [6]:
# Set model2 parameters
rail.set_model2(timelimit=60, verbose=True)

# Set constraints
rail.set_constraints()

# Set objective
rail.set_objective()

Set parameter TimeLimit to value 60
Set parameter PoolSolutions to value 1
Set parameter Cuts to value 0
Set parameter CutPasses to value 0
Set parameter Heuristics to value 0
Set parameter Symmetry to value 0
Set parameter Threads to value 1
Set parameter Presolve to value 0
Set parameter NumericFocus to value 3


For this model we then set the valid inequalities to be used in the optimization processand the cutting planes generators with the dedicated implemented methods.

In [7]:
# Set valid inequalities
rail.set_valid_inequalities()

# Set cuttings planes
rail.set_cutting_planes()

Set parameter BQPCuts to value -1
Set parameter CliqueCuts to value -1
Set parameter CoverCuts to value -1
Set parameter FlowCoverCuts to value -1
Set parameter FlowPathCuts to value -1
Set parameter GomoryPasses to value -1
Set parameter GUBCoverCuts to value -1
Set parameter ImpliedCuts to value -1
Set parameter LiftProjectCuts to value -1
Set parameter MIRCuts to value -1
Set parameter ModKCuts to value -1
Set parameter NetworkCuts to value -1
Set parameter RelaxLiftCuts to value -1
Set parameter StrongCGCuts to value -1
Set parameter ZeroHalfCuts to value -1


Finally the optimization can be run with the `optimize` method as always.

In [8]:
# Solve the scheduling problem
rail.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Non-default parameters:
TimeLimit  60
Heuristics  0
Symmetry  0
Cuts  0
CutPasses  0
NumericFocus  3
Presolve  0
Threads  1
PoolSolutions  1

Optimize a model with 9755 rows, 4600 columns and 28839 nonzeros
Model fingerprint: 0x6e71ca5e
Variable types: 1350 continuous, 3250 integer (3250 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+05]
  Objective range  [4e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+05]

Loaded user MIP start with objective 3913.47

Variable types: 1350 continuous, 3250 integer (3250 binary)

Root relaxation: objective -9.208446e+04, 513 iterations, 0.13 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

{'runtime': 0.6067149639129639,
 'nodes': 3.0,
 'iterations': 590.0,
 'gap': 0.0,
 'obj': 3067.026331473724}